In [ ]:
import urllib
import numpy as np
import tensorflow as tf
from rcnn_sat import preprocess_image
import os
import json
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy
from scipy import stats
import scipy.io as sio

# Define the train and validation data

In [ ]:
#load and preprocess training & validation data
#training
with open('2400_selected_scenes_places365_train_standard.json') as json_file:
    subset_scenes_dict_train = json.load(json_file)  
train_set_path = '/scratch/agnek95/PDM/places_365_256_train_val/data_256/'
train_image_paths = list(subset_scenes_dict_train.keys())
train_imgs_prep = np.ones([len(train_image_paths),128,128,3])
train_imgs_prep[:] = np.nan
for idx,image_path in enumerate(train_image_paths):
        image = load_img(train_set_path+image_path, target_size=(128, 128)) 
        image = img_to_array(image)
        image = np.uint8(image)
        image = preprocess_image(image)
        train_imgs_prep[idx,:,:,:] = image
        
train_images_paths = [train_set_path+image_path for image_path in train_image_paths]

#validation    
with open('1200_selected_scenes_places365_val_standard.json') as json_file:
    subset_scenes_dict_val = json.load(json_file)  
val_set_path = '/scratch/agnek95/PDM/places_365_256_train_val/val_256/' 
val_image_paths = list(subset_scenes_dict_val.keys())
val_imgs_prep = np.ones([len(val_image_paths),128,128,3])
val_imgs_prep[:] = np.nan
for idx,image_path in enumerate(val_image_paths):
        image = load_img(val_set_path+image_path, target_size=(128, 128)) 
        image = img_to_array(image)
        image = np.uint8(image)
        image = preprocess_image(image)
        val_imgs_prep[idx,:,:,:] = image



# Define the test data 

In [ ]:
main_path = '/home/agnek95/SMST/PDM_PILOT_2/ANALYSIS_Full_experiment/DNN/'
data_path = '/scratch/agnek95/PDM/places365/val_large'
#IDs of the scenes of interest
with open(os.path.join(main_path,'scenes_eeg_ordered.json')) as json_file:
    scenes_60 = json.load(json_file)      
         
selected_scenes = list(scenes_60.keys())
test_images_paths = [None]*len(selected_scenes)
for index,file in enumerate(selected_scenes):
    test_images_paths[index] = os.path.join(data_path,selected_scenes[index])

In [ ]:
#labels: man-made: 0, natural: 1
y_train = np.array([label for label in range(2) for reps in range(int(train_imgs_prep.shape[0]/2))]) 
y_val = np.array([label for label in range(2) for reps in range(int(val_imgs_prep.shape[0]/2))])

x_train =  train_imgs_prep
x_val = val_imgs_prep

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)


# Import the network graph with readouts (one readout at a time), load weights from previously fine-tuned network (on scenes) w/o readouts and collect predictions

In [ ]:
def train_readout(model):
    base_learning_rate = 0.0001 #in case you want to use optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    history = model.fit(
        x_train,
        y_train,
        shuffle=True,
        batch_size=10,
        epochs=20, 
        validation_data=(x_val,y_val)
    )


In [ ]:
def collect_predictions(model,num_images_all, num_classes, batch_size):
    num_batches = int(num_images_all / batch_size)
    pred = np.ones([num_batches,batch_size,num_classes])
    pred[:] = np.nan

    #preprocess images
    for batch, img_idx in enumerate(range(0, num_images_all, batch_size)):
        batch_paths = test_images_paths[img_idx:img_idx + batch_size] 
        batch_images = np.zeros((batch_size,128,128,3)) 
        for i, image_path in enumerate(batch_paths):
            image = load_img(image_path, target_size=(128, 128)) 
            image = img_to_array(image)
            image = np.uint8(image)
            image = preprocess_image(image)
            batch_images[i,:,:,:] = image

        #predictions
        pred[batch,:,:] = model(batch_images)

    #reshape: all images from all batches in one dimension
    # pred_reshaped =  np.transpose(pred,(0,2,1,3)).reshape(num_batches*batch_size,num_classes)
    pred_reshaped =  pred.reshape(num_batches*batch_size,num_classes)
    
    return pred_reshaped

In [ ]:
def non_trainable(model: tf.keras.Model):
   for layer in model.layers:
            if 'Readout' in layer.name:
                layer._trainable = True
            else:
                layer._trainable = False
     

## One readout at a time - load the weights of the other layers, train the readout and get the predictions

In [ ]:
model_type = 'b_d' 

In [ ]:
from rcnn_sat import b_d_net_readout_0, b_d_net_readout_1, b_d_net_readout_2, b_d_net_readout_3, b_d_net_readout_4, b_d_net_readout_5, b_d_net_readout_final

input_layer = tf.keras.layers.Input((128, 128, 3))
weights_name='{}_net_scenes_weights.h5'.format(model_type)
num_readouts=7
num_classes=2
num_images_all = len(test_images_paths)
all_models = [b_d_net_readout_0, b_d_net_readout_1, b_d_net_readout_2, b_d_net_readout_3, b_d_net_readout_4, b_d_net_readout_5, b_d_net_readout_final]
# all_models = [b_d_net_readout_0, b_d_net_readout_1]
predictions_all_readouts = np.zeros([num_readouts,num_images_all,num_classes])
predictions_all_readouts[:] = np.nan

for m,readout_model in enumerate(all_models):
    tf.keras.backend.clear_session() 
    
    model = readout_model(input_layer,classes_scenes=2)
    non_trainable(model)
    # tf.keras.backend.set_floatx('float64') #To change all layers to have dtype float64 by default - warning from TF
    # print(model.summary())
    
    #Load weights
    model.load_weights(weights_name,by_name=True)
    
    #train the readout
    train_readout(model)

    #save weights
    model_name='{}_net_scenes_readout_{}'.format(model_type,m)
    model.save_weights(model_name+'_weights.h5')

    #collect predictions
    batch_size = 60
    predictions_all_readouts[m,:,:]=collect_predictions(model,num_images_all, num_classes, batch_size)
    



# Get RTs

In [ ]:
def get_RTs(entropy_thresh):
    #get entropies for each image & each timepoint
    entropies_pred = np.ones([num_readouts, num_images_all])
    entropies_pred[:] = np.nan

    for readout in range(num_readouts):
        for image in range(num_images_all):
            entropies_pred[readout,image] = entropy(predictions_all_readouts[readout,image,:])

    #for each image, determine the timepoint when entropy reaches threshold
    rt_thresh = np.ones(num_images_all)
    rt_thresh[:] = np.nan
    
    for image in range(num_images_all):
        for readout in range(num_readouts):
            if entropies_pred[readout,image] <= entropy_thresh:
                rt_thresh[image]=readout
                break          

    #if it never reaches the threshold (nan in the array), replace by 8
    rt_thresh[np.isnan(rt_thresh)] = 8
   
    return rt_thresh,entropies_pred

In [ ]:
#load RTs
rts_eeg_dict = sio.loadmat('RT_all_subjects_5_35_categorization.mat')
rts_eeg = rts_eeg_dict.get('RTs')

#define some variables
num_subjects = rts_eeg.shape[0]
entropies = np.arange(0.01,0.1,0.01)
best_entropy = np.ones([num_subjects])
best_entropy[:] = np.nan
correlation_test = np.ones([num_subjects,3]) #all,artificial,natural
correlation_test[:] = np.nan
num_scenes = len(test_images_paths)

#get RNN RTs for every entropy threshold and correlate with humans
rts_dnn = np.ones([len(entropies),len(test_images_paths)])
rts_dnn[:] = np.nan

for idx,e in enumerate(entropies):
    rts_dnn[idx,:],entropies_pred = get_RTs(e)
    
    
#for each fold, fit the entropy threshold on 29 subjects
for s in range(num_subjects): 
    artificial_idx = np.arange(30)
    natural_idx = np.arange(30,60)

    test_sub = rts_eeg[s,:]
    fit_sub = np.nanmean(rts_eeg[np.arange(num_subjects)!=s,:],0)
    correlation_fit = np.ones([len(entropies),2])
    correlation_fit[:] = np.nan

    for idx,e in enumerate(entropies):
        correlation_fit[idx,0] = stats.pearsonr(np.squeeze(rts_rnn[idx,artificial_idx]),fit_sub[artificial_idx])[0] #artificial
        correlation_fit[idx,1] = stats.pearsonr(np.squeeze(rts_rnn[idx,natural_idx]),fit_sub[natural_idx])[0] #natural
        
    #select the entropy with highest mean (over scene conditions) correlation
    best_entropy[s] = round(entropies[np.nanargmax(np.mean(correlation_fit,1))],2)     
    idx_best_entropy=np.nanargmax(np.mean(correlation_fit,1))
    
    #remove scene if there's no RT for it 
    selected_dnn_rts = rts_dnn[idx_best_entropy,:]
    if np.argwhere(np.isnan(test_sub)).size:
        print(s)
        removed_scene = np.argwhere(np.isnan(test_sub))[0][0]
        if removed_scene in natural_idx:
            natural_idx = np.delete(natural_idx,removed_scene-30)
        elif removed_scene in artificial_idx:
            artificial_idx = np.delete(artificial_idx,removed_scene)

    #correlate with leftout subject        
    correlation_test[s,0] = stats.pearsonr(selected_dnn_rts[np.concatenate((artificial_idx,natural_idx))],\
                                           test_sub[np.concatenate((artificial_idx,natural_idx))])[0]        
    correlation_test[s,1] = stats.pearsonr(selected_dnn_rts[artificial_idx],test_sub[artificial_idx])[0]
    correlation_test[s,2] = stats.pearsonr(selected_dnn_rts[natural_idx],test_sub[natural_idx])[0]
    
print(best_entropy)
print(np.mean(correlation_test,0))
RT_entropy = stats.mode(best_entropy)[0][0] #which should hopefully be the same for every fold anyway - idx_best_entropy
np.save('correlation_RT_human_{}_net_7_layers_cross-validated_readouts'.format(model_type),correlation_test)
np.save('{}_net_RTs_7_layers_entropy_threshold_readouts_{}'.format(model_type,RT_entropy),selected_dnn_rts)

#save as mat files
sio.savemat('correlation_RT_human_{}_net_7_layers_cross-validated_readouts.mat'.format(model_type),dict(data=correlation_test))
sio.savemat('{}_net_RTs_7_layers_entropy_threshold_readouts_{}.mat'.format(model_type,RT_entropy),dict(data=selected_dnn_rts))


In [ ]:
# #save weights
# model_name='{}_net_scenes_readout_{}'.format(model_type,m)
# print(model_name)
# model.save_weights(model_name+'_weights.h5')

In [ ]:
# from rcnn_sat import b_d_net_readouts
# input_layer = tf.keras.layers.Input((128, 128, 3))
# model = b_d_net_readouts(input_layer, classes_scenes=2)

## Load the weights of all layers but the readouts

In [ ]:
# weights_name='{}_net_scenes_weights.h5'.format(model_type)
# model.load_weights(weights_name,by_name=True)

In [ ]:
# from rcnn_sat import b_d_net_readout_final
# model = b_d_net_readout_final(input_layer, classes_scenes=2)
# model.summary()

## Define trainable layers (readouts)

In [ ]:
# for layer in model.layers:
#     if 'Readout' in layer.name:
#         layer.trainable = True
#     else:
#         layer.trainable = False
               
# for layer in model.layers:
#     if layer.trainable==True:
#         print(layer.name, layer.trainable)

In [ ]:
# num_readouts=7
# x_train_all = np.repeat(x_train[None,...],7,axis=0)
# y_train_all = np.tile(y_train,(num_readouts,1))

# x_val_all = np.repeat(x_val[None,...],7,axis=0)
# y_val_all = np.tile(y_val,(num_readouts,1))

# print(x_train_all.shape)
# print(y_train_all.shape)
# print(x_val_all.shape)
# print(y_val_all.shape)



In [ ]:
# base_learning_rate = 0.0001 #in case you want to use optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate)
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#               metrics=['accuracy'])

# history = model.fit(
#     (x_train,x_train,x_train,x_train,x_train,x_train,x_train),
#     (y_train,y_train,y_train,y_train,y_train,y_train,y_train),
#     shuffle=True,
#     batch_size=10,
#     epochs=20, #5?
#     validation_data=((x_val,x_val,x_val,x_val,x_val,x_val,x_val), (y_val,y_val,y_val,y_val,y_val,y_val,y_val)),
)


In [ ]:
# y_pred = model.predict(x_val, batch_size = 10)
# print(y_pred)

## Save the weights

In [ ]:
# model_name='{}_net_scenes_readouts'.format(model_type)
# model.save_weights(model_name+'_weights.h5')

# If you want to get the RTs rightaway, follow the next steps, otherwise use the separate script (rnn_dth_collect_activations.ipynb)

In [ ]:
# def get_RTs(test_images_path,batch_size,entropy_thresh):
#     num_images_all = len(test_images_path)
#     num_batches = int(num_images_all / batch_size)
#     num_timepoints = 8
#     num_classes = 2
#     all_batches_activ = np.ones([num_batches, batch_size, 128, 128, 3])
#     all_batches_activ[:] = np.nan
#     pred = np.ones([num_batches,num_timepoints,batch_size,num_classes])
#     pred[:] = np.nan

#     for batch, img_idx in enumerate(range(0, num_images_all, batch_size)):
#         batch_paths = test_images_path[img_idx:img_idx + batch_size] 
#         batch_images = np.zeros((batch_size,128,128,3)) 
#         for i, image_path in enumerate(batch_paths):
#             image = load_img(image_path, target_size=(128, 128)) 
#             image = img_to_array(image)
#             image = np.uint8(image)
#             image = preprocess_image(image)
#             batch_images[i,:,:,:] = image

#         #predictions
#         pred[batch,:,:,:] = model(batch_images) #shape: num_timepoints x batch_size x classes

#     #reshape: all images from all batches in one dimension
#     pred_reshaped =  np.transpose(pred,(0,2,1,3)).reshape(num_batches*batch_size,num_timepoints,num_classes)

    #get entropies for each image & each timepoint
#     entropies_pred = np.ones([num_images_all,num_timepoints])
#     entropies_pred[:] = np.nan

#     for image in range(num_images_all):
#         for tp in range(num_timepoints):
#             entropies_pred[image,tp] = entropy(pred_reshaped[image,tp])

#     # #for each image, determine the timepoint when entropy reaches threshold
#     rt_thresh = np.ones(num_images_all)
#     rt_thresh[:] = np.nan
#     for image in range(num_images_all):
#         for tp in range(num_timepoints):
#             if entropies_pred[image,tp] <= entropy_thresh:
#                 rt_thresh[image]=tp
#                 break          

#     #if it never reaches the threshold (nan in the array), replace by 8
#     rt_thresh[np.isnan(rt_thresh)] = 8
   
#     return rt_thresh